NLP y Visualización

Input: Datasets Predios, Publicaciones Científicas, Nombre Especies, Países
Output: Análisis de tendencias

In [1]:
#importar librerias

import json
import spacy
import en_core_web_sm
#from spacy import displacy

import pandas as pd
import plotly_express as px
#from spacy.tokens import DocBin
from collections import Counter

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

import re

import dash
from dash import dcc, html, Input, Output, dash_table,callback, Dash

from dash.exceptions import PreventUpdate
from collections import defaultdict
from PIL import Image
import dash_bootstrap_components as dbc

Cargar datasets: 
Predios, Publicaciones Científicas, Nombre Especies y Países

In [9]:
# Dataset predios
df_predios=pd.read_excel('..\Archivos\Terrenos.xlsx',sheet_name='Sheet1')

#Dataset Publicaciones Científicas
with open ('..\Archivos\pubmed_data262.json','r',encoding='utf-8') as f:
    pubmed_data=json.load(f)

#Dataset Nombre Especies
df_especies=pd.read_excel('..\Archivos\Terrenos.xlsx',sheet_name='Hoja1')

#Dataset Países
codigo_paises=pd.read_excel('..\Archivos\Copia de código paises.xlsx')

<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:5: SyntaxWarning: invalid escape sequence '\A'
<>:9: SyntaxWarning: invalid escape sequence '\A'
<>:12: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:5: SyntaxWarning: invalid escape sequence '\A'
<>:9: SyntaxWarning: invalid escape sequence '\A'
<>:12: SyntaxWarning: invalid escape sequence '\A'
C:\Users\56985\AppData\Local\Temp\ipykernel_11024\3615361628.py:2: SyntaxWarning: invalid escape sequence '\A'
  df_predios=pd.read_excel('..\Archivos\Terrenos.xlsx',sheet_name='Sheet1')
C:\Users\56985\AppData\Local\Temp\ipykernel_11024\3615361628.py:5: SyntaxWarning: invalid escape sequence '\A'
  with open ('..\Archivos\pubmed_data262.json','r',encoding='utf-8') as f:
C:\Users\56985\AppData\Local\Temp\ipykernel_11024\3615361628.py:9: SyntaxWarning: invalid escape sequence '\A'
  df_especies=pd.read_excel('..\Archivos\Terrenos.xlsx',sheet_name='Hoja1')
C:\Users\56985\AppData\Local\Temp\ipyk

In [12]:
# Cargar GeoJSON
with open('..\\Archivos\\terrenos_enriquecido_ID.geojson', "r", encoding="utf-8") as f:
    geojson_data = json.load(f)

In [13]:

df_predios=pd.merge(df_predios,df_especies[['ID_ESP1','ID','Nombre científico']])
df_predios_group=df_predios.groupby(['USO_CEFOR','Nombre científico','ID'])['SUP_HA'].sum().reset_index()
df_predios_group_lplp=df_predios.groupby(['NOMBRE','Nombre científico','ID'])['SUP_HA'].sum().reset_index()

In [15]:
ruta_imagen=Image.open('..\Archivos\Logo_udec_maci.png')
#ruta_imagen_predios=Image.open('Terrenos.png')

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\56985\AppData\Local\Temp\ipykernel_11024\3546564186.py:1: SyntaxWarning: invalid escape sequence '\A'
  ruta_imagen=Image.open('..\Archivos\Logo_udec_maci.png')


In [16]:
#wordcloud

def procesar_keywords (keywords):

    kw=[]
    for keyword in keywords:
        for key in keyword:
            kw.append(key)

    keywords_string=' ,'.join(e for e in kw)
    return keywords_string


NPL y Visualización

In [17]:
import plotly.graph_objects as go

In [18]:
with open('..\Archivos\limites_unificados.geojson', "r", encoding="utf-8") as f:
    geojson_limite = json.load(f)

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\56985\AppData\Local\Temp\ipykernel_11024\3952774604.py:1: SyntaxWarning: invalid escape sequence '\A'
  with open('..\Archivos\limites_unificados.geojson', "r", encoding="utf-8") as f:


In [19]:
#probando esto

app=dash.Dash()

#funciones globales:

nlp = spacy.load("en_core_web_sm")     #cargo el modelo

options_dict=dict(zip(df_especies['Nombre científico'],df_especies['ID']))     #defino las opciones del menú desplegable

def clean_text(texto):                                                         #defino la función de limpieza del texto
    return re.sub(r"<.*?>|[%()=/-]|\d", "", str(texto)) #nueva versión
    #return re.sub("<.*?>|%|\(|\)|=|-/n|/>-|\d", "", str(texto))

def procesar_json_fil(diccionario):                                         #esta es la función de procesamiento principal
    lista=[]
    data={}

    for entry in diccionario:#lista-dicc. accedo al primer elemento
        
        for doc_id , text in entry.items():
            doc = nlp(clean_text(text))                                      #llamo a la función de limpieza
            data_id=[]
            for token in doc:
                if token.pos_ in ["NOUN","PROPN"] and len(token.lemma_)>3 and not token.is_stop:
                    
                    data_id.append(token.lemma_.lower())
                data[doc_id]=data_id
    lista.append(data)                                                      #parece que esta parte está un poco demás
  
    #lista de todas las claves
    M=[]
    for x,y in data.items():
        for e in y:
            M.append(e)
    L=list(set(M))
    #procesamiento de frecuencias
    filas=[]
    pal_tot=len(M)
    lis_tot=len(diccionario)

    for e in L:
        total=0
        listas_presentes=[]

        for x,y in data.items():
            count = y.count(e)
            total += count
            if count>0:
                listas_presentes.append(x)

        filas.append([e,listas_presentes,total/pal_tot*100,len(listas_presentes),len(listas_presentes)/lis_tot*100,total])
        
    df_res_res=pd.DataFrame(filas,columns=["lemma","listas presentes","f abs","cantidad listas","f rel","total"])
    return df_res_res

#Procesar json: obtener países

#probar generando una lista de dicc

def procesar_paises (valor):

    #valor=[31,18,25]
    lista_pmids=[]

    for paper in pubmed_data:
        if paper['ID'] in valor:
            lista_pmids.append(paper['PMID'])

    #aff=[]
    #pm=[]
    A={}
    D=[]
    #Lista=[]
    for record in pubmed_data:
        if record['PMID'] in lista_pmids:
                    
            for autor in record['Authors']:
                affi=autor['Affiliations']
                for inst in affi:
                
                    pm2="".join(record['PMID'])
                    aff2="".join(inst)
                    A[pm2]=aff2
                    D.append(A)


    #lista=[]
    data=[]

    for entry in D:
        for doc_id, text in entry.items():

            doc=nlp(clean_text(text))
            paises=[ent.text for ent in doc.ents if ent.label_ =="GPE"]
            for pais in set(paises):
                    data.append({"PMID":doc_id,"GPE":pais})

    df_p=pd.DataFrame(data)

    #df_p["PMID"] = pd.to_numeric(df_p["PMID"],errors="coerce")
    df_p_unique=df_p.drop_duplicates(subset=["PMID","GPE"])

    df_unique_conteo=df_p_unique['GPE'].value_counts().reset_index()
    df_unique_conteo.columns=['País','Recuento']
    df_unique_conteo_codigo=pd.merge(df_unique_conteo,codigo_paises,left_on="País",right_on="Country")

    return df_unique_conteo_codigo



#Inicio app

app=Dash(__name__,external_stylesheets=[dbc.themes.LUX,dbc.icons.FONT_AWESOME])

app.layout=dbc.Container([
    html.Br(),
    dbc.Row([dbc.Col([html.Img(src=ruta_imagen,style={'height':'40px','width':'10%'})])]),
    html.Br(),
    html.H4("Tesina Beatriz Ugalde - 2025"),
    html.Br(),
    html.Br(),
    html.H1("Identificar líneas de trabajo para Potenciar Investigación y Docencia en predios forestales",style={'text-align':'center','margin-top':'20px'}),
    html.Br(),
    html.Br(),
    html.Br(),
    html.P("Se busca potenciar la investigación y docencia en predios forestales. "

"Se muestra la caracterización de dos predios en términos de composición y  uso de suelo. " 

"Se realiza una búsqueda de publicaciones científicas relacionadas con las especies que los habitan, se extraen términos relevantes y se analiza su importancia en función de su presencia y abundancia, promoviendo la búsqueda de tendencias innovadoras en investigación científica.",style={'text-align':'justify'}),
    html.Br(),
    html.Br(),
    html.H4("Caracterización predios forestales"),
    html.Br(),
    html.P("El centro forestal (CEFOR) perteneciente a la Universidad Austral de Chile, cuenta con un patrimonio de más de 4000 hectáreas de predios forestales, donde se realizan labores de docencia, investigación y extensión. Dos de los predios más antiguos, “Los Pinos“ y “Las Palmas“ cuentan con cerca de 1500 hectáreas de terreno, "
    "están ubicados a 5 Km del Campus Isla Teja, y presentan gran valor biológico. "  
    " Seleccione una o más especies para conocer su ubicación, uso y potenciales líneas de investigación", style={'text-align':'justify'}),
    #html.H6("Seleccione una o más especies"),
    html.Br(),
    html.Div([ dcc.Dropdown(options=[{'label':NOM,'value':ID} for NOM,ID in sorted(options_dict.items(),key=lambda x: x[1], reverse=True)],id="entrada",multi=True)]),
    #html.Div([dcc.Input(),dcc.Input(style={'margin_left':'15px'})]),
    html.Br(),
    html.H6("Descripción de las especies seleccionadas"),
    html.Br(),
    dbc.Row([dbc.Col([dash_table.DataTable(id='tabla_nombres',style_cell={'textAlign':'left'})]),dbc.Col([dcc.Graph(id='mapa-terrenos', style={"height": "80vh"})])]),
    html.Br(),
    html.Br(),
    #html.Div([dcc.Graph(id='mapa-terrenos', style={"height": "80vh"})]),
    #html.H6("Relación de las especies con los predios"),
    dbc.Row([
        dbc.Col([dcc.Graph(id='predios_barras',style={'height':'400px','width':'100%'})]),dbc.Col([dcc.Graph(id='predios_barras_lplp',style={'height':'400px','width':'100%'})])]),
    html.Br(),
    html.Br(),
    html.H4("Extracción de términos desde publicaciones científicas"),
    html.Br(),
    html.P("A partir de las especies seleccionadas en la parte superior, Se obtienen publicaciones científicas desde pubmed central si la especie aparece en el título o abstract." 
"Se muestra una nube de palabras que agrupa todas las “keywords“ de las publicaciones seleccionadas,  la evolución de las publicaciones en el tiempo y los países donde se investigan dichas especies."),
    html.Br(),
            
    dbc.Row([
        dbc.Col([html.Img(id='wordcloud',style={'height':'100%','width':'95%'})]),
        dbc.Col([dcc.Graph(id='paper_año',style={'height':'100%','width':'95%'})]),
        dbc.Col([dcc.Graph(id='mapa_paises',style={'height':'100%','width':'95%'})])],style={'height':'400px'}),
    html.Br(),
    html.Br(),    
    html.P("A continuación se realiza un análisis semántico de los abstracts y se extraen términos importantes  y se analizan en función de su presencia y abundancia. La presencia se determina evaluando en cuántas publicaciones diferentes aparece cada término, y la abundancia cuántas veces aparece en total. Así, un término altamente presente y abundante se espera que sea de poca importancia, ya que es transversal a todas las publicaciones, ahora bien un término poco presente podría representar una línea de trabajo innovadora."),

    dcc.Graph(id='grafico'),
    html.Br(),
    html.H6("Información relacionada al término seleccionado:"),
    html.Br(),
    dbc.Row([dbc.Col([dash_table.DataTable(id='tabla',style_cell={'textAlign':'left'},style_data={'whiteSpace':'normal','width':'100px','maxWidth':'100px','minWidth':'100px','height':'auto'},style_cell_conditional=[{'if':{'column_id':'Título'},'width':'20%'},{'if':{'column_id':'Institución'},'width':'20%'}])])]),
    html.Br(),
    html.P("Para más información, visite la URL")
    ])
        

#aquí debería ir el callback del tipo de especies: va a limitar las opciones que aparezcan a continuación

@callback(
        Output("predios_barras","figure"),
        Output("predios_barras_lplp","figure"),
        Output("tabla_nombres","data"),
        Output("tabla_nombres","columns"),
        Output('wordcloud','src'),
        Output('paper_año','figure'),
        Output('mapa_paises','figure'),
        Output("grafico","figure"),
        Output("mapa-terrenos", "figure"),
        Input("entrada","value"))

#Primero filtrar el JSON por especie

def filtro(valor):

    if valor is None:  # Si no hay selección
        raise PreventUpdate
    else:
        #Filtrar por nombre/ID
        df_especies_filtrado = df_especies[df_especies['ID'].isin(valor)]
        nombre_cien= df_especies_filtrado['Nombre científico']   
        nombre_com= df_especies_filtrado['Nombre común']
        nombre_ori=df_especies_filtrado['Nombre originario']         
        df=pd.DataFrame({'Nombre científico':nombre_cien,'Nombre común':nombre_com,'Nombre originario':nombre_ori})
        data=df.to_dict('records')
        columns=[{'id':c,'name':c}for c in df.columns]

        #esta parte filtra el json: abstract
        json_filtrado = [{paper['PMID']: paper['Abstract']} for paper in pubmed_data if paper['ID'] in valor]
        fig=px.scatter(procesar_json_fil(json_filtrado),x='f rel',y='f abs',size='total',hover_data=['listas presentes'],hover_name='lemma',opacity=0.5,title="Presencia y Abundancia de términos",labels={'f rel':'Presencia','f abs':'Abundancia'})

        #esta parte filtra el json: solo pmids:lista
        lista_pmids=[]
        for paper in pubmed_data:
            if paper['ID'] in valor:
                lista_pmids.append(paper['PMID'])

        #Supongo que aquí tiene que ir para el mapa mundi
        # 
        
        
        #llamar a función paises y graficar mapa mundi
        
        #keywords y año
        keywords=[]
        año_paper=[]
        pubmedid=[]
        nombrecien=[]

        for paper in pubmed_data:
            if paper['ID'] in valor:
                keywords.append(paper['Keywords'])
                año_paper.append(paper['Date']['Year'])
                pubmedid.append(paper['PMID'])
                nombrecien.append(paper['Topic'])
        wc=WordCloud(background_color='white',colormap='Greens',width=400,height=400).generate(procesar_keywords(keywords))
        wc.to_file("wordcloudkw.png")
        encoded_image= Image.open('wordcloudkw.png')
        
        #gráfico papers por año
        df_paper_año=pd.DataFrame({'PMID':pubmedid,"Año":año_paper,"Nombre científico":nombrecien})
        df_paper_año=df_paper_año.groupby(['Nombre científico','Año'])['PMID'].count().reset_index()
        fig_pap_año=px.bar(df_paper_año,x="Año",y="PMID",color='Nombre científico',labels={'PMID':'Número de Publicaciones'},title='Número de papers publicados por año',width=400,height=400)
        fig_pap_año.update_layout(xaxis={'categoryorder':'category ascending'},margin={"r":10,"t":10,"l":10,"b":10})

        fig_mapa=px.choropleth(procesar_paises(valor),locations="Alpha-3 code",color="Recuento",color_continuous_scale='Greens',projection='natural earth',title='Países donde se investiga',width=400,height=400)
        fig_mapa.update_geos(showcoastlines=False,framewidth=0.3,framecolor="#444")
        fig_mapa.update_layout(coloraxis_showscale=True)
        #fig_mapa.update_layout()


        #filtro el dataset predios
        df_predios_group_filtro = df_predios_group[df_predios_group['ID'].isin(valor)]
        fig_predios=px.bar(df_predios_group_filtro,x='USO_CEFOR',y='SUP_HA',color='Nombre científico',text_auto=True,labels={'SUP_HA':'Superficie (Há)','USO_CEFOR':'Uso de suelo'},title='Superficie por uso de suelo y especie')
        fig_predios.update_layout(xaxis={'categoryorder':'total descending'})

        df_predios_group_filtro_lplp = df_predios_group_lplp[df_predios_group_lplp['ID'].isin(valor)]
        fig_predios_lplp=px.bar(df_predios_group_filtro_lplp,x='NOMBRE',y='SUP_HA',color='Nombre científico',text_auto=True,labels={'SUP_HA':'Superficie (Há)','NOMBRE':'Predio'},title='Superficie por predio y especie')
        fig_predios_lplp.update_layout(xaxis={'categoryorder':'total descending'})

    #mapa coropletico

    if not valor:
        features_filtrados = geojson_data["features"]
    else:
        features_filtrados = [
            f for f in geojson_data["features"]
            if f["properties"]["ID_es"] in valor
        ]

        for i, f in enumerate(features_filtrados):
            f["id"] = f.get("id", str(i))

    geojson_filtrado = {
        "type": "FeatureCollection",
        "features": features_filtrados
    }

    # Crear figura vacía
    fig_2 = go.Figure()

    # ➤ Capa base: límites del predio (línea negra)
    # ➤ Capa base: límites del predio (líneas negras)
    for feature in geojson_limite["features"]:
        if feature["geometry"]["type"] == "MultiPolygon":
            for polygon in feature["geometry"]["coordinates"]:
                for ring in polygon:
                    lons, lats = zip(*ring)
                    fig_2.add_trace(go.Scattermap(
                        lon=lons,
                        lat=lats,
                        mode="lines",
                        line=dict(width=2, color="black"),
                        name="Límite predio",
                        hoverinfo="skip",
                        showlegend=False
                    ))
        elif feature["geometry"]["type"] == "Polygon":
            for ring in feature["geometry"]["coordinates"]:
                lons, lats = zip(*ring)
                fig_2.add_trace(go.Scattermap(
                    lon=lons,
                    lat=lats,
                    mode="lines",
                    line=dict(width=2, color="black"),
                    name="Límite predio",
                    hoverinfo="skip",
                    showlegend=False
                ))

    # ➤ Capa de especies: usando choropleth
    fig_map = px.choropleth_map(
        geojson=geojson_filtrado,
        locations=[f["id"] for f in features_filtrados],
        color=[f["properties"]["Clasificación"] for f in features_filtrados],
        hover_name=[f["properties"]["Nombre científico"] for f in features_filtrados],
        featureidkey="id",
        zoom=13,
        center={"lat": -39.74, "lon": -73.14},
        #opacity=0.5
    )

    # ➤ Añadir la capa de especies sobre la figura existente
    for trace in fig_map.data:
        fig_2.add_trace(trace)

    # Ajustar el diseño final
    fig_2.update_layout(
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        map=dict(
            style="open-street-map",
            zoom=13,
            center={"lat": -39.74, "lon": -73.14}
        )
    )        
        
    return fig_predios,fig_predios_lplp,data,columns,encoded_image,fig_pap_año,fig_mapa,fig,fig_2

#segundo callback

@callback(
    Output('tabla','data'),
    Output('tabla','columns'),
    Input('grafico','clickData')
    )

def display_click_data(clickData):
    if clickData is None:  # Si no hay selección
        raise PreventUpdate
    else:
    
       #id=[p['custDataomdata'][0] for p in selectedData['points']]
       id=clickData['points'][0]['customdata'][0] #lista
       r=[]
       for item in pubmed_data:
           if item['PMID']in id:
               r.append(item)
       #aquí tengo que generar el df, no puedo trabajar con isin pq tengo json
       pmid=[]
       titulo=[]
       urla=[]
       revista=[]
       fecha=[]
       autor=[]
       inst=[]
       for paper in r:
           pmid.append(paper['PMID'])
           titulo.append(paper['Title']) #aquí se edita el formato del título? json.dumps?
           urla.append(paper['URL'])
           revista.append(paper['Journal'])
           fecha.append(paper['Date']['Month']+"/"+paper['Date']['Year'])
           autor.append(paper['Authors'][0]['Name']+" "+paper['Authors'][0]['Lastname'])
           inst.append("".join(paper['Authors'][0]['Affiliations']))

           df=pd.DataFrame({'PMID':pmid,'Título':titulo,'1ª autor':autor,'Institución':inst,'URL':urla,'Revista':revista,'Año':fecha})#creo que no es necesario? tal vez para hacer una tabla

           data=df.to_dict('records')
           columns=[{'id':c,'name':c}for c in df.columns]

                  
    return data,columns
    


if __name__=='__main__':
    app.run_server(port=8053,use_reloader=False,debug=True)